In [1]:
# import needed modules
import pandas as pd
import os
import scipy.stats as st
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
# data set csv file variables
mouseData = ("Mouse_metadata.csv")
studyData = ("Study_results.csv")

In [3]:
# read the data sets
mouseMetadata = pd.read_csv(mouseData)
studyResults = pd.read_csv(studyData)

In [4]:
# merge data sets 
raw_data = pd.merge(mouseMetadata, studyResults, on="Mouse ID")

In [5]:
raw_data.head()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
